In [1]:
from lime.lime_text import LimeTextExplainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

import torch
import torch.nn.functional as F
import scipy as sp
import numpy as np

import re
import shap

C:\Users\Tazeek\anaconda3\envs\action-lab_projects\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# SenticNet

- Break a statement into sentences.
- Find the polarity per sentence.
- Find the polarity of the statement.

In [ ]:
# Define a regular expression pattern to split based on punctuations
split_pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)'

# Remove non-alphabets
regex = re.compile('[^a-zA-Z]')

In [ ]:
# Load SenticNet variable
from senticnet import senticnet

# First step: Lowercase the statement
# Second step: Get the key (if it is there)
# Third step: Get the polarity value and polarity type
# Fourth step: Sum up the polarities in the sentence

print(senticnet['happy'][6])
print(senticnet['happy'][7])

In [ ]:
def fetch_polarities(words_list, senticnet):

    sentence_polarity = {}
    
    # Loop one at a time
    for word in words_list:

        # Check if the word exists
        # If it does not, add 0 as the polarity value
        if word not in senticnet:
            continue
            
        sentence_polarity[word] = senticnet[word][7]

    # Return
    return sentence_polarity

random_sentence = "Hello world! I am really happy to share something with you, despite going through some rough time."

# Split the text into sentences based on the defined pattern
sentences = re.split(split_pattern, random_sentence)

# Loop one sentence at a time
for sentence in sentences:
    
    # Remove spacing
    sentence = sentence.strip()
    print(sentence)

    # Split by word
    words = sentence.split(" ")

    # Clean up
    words = [regex.sub('', word.lower()) for word in words]

    # Fetch the polarities per sentence
    sentence_polarity = fetch_polarities(words, senticnet)
    
    print(sentence_polarity)
    print("\n\n")

# LIME

This section is for intrepreting models, using LIME

In [3]:
# For loading
model_name = "nateraw/bert-base-uncased-emotion"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, do_lower_case=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, output_hidden_states=True)
labels = sorted(model.config.label2id, key=model.config.label2id.get)

class_names = labels
print(labels)

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [ ]:
# Create the function to fetch the probability scores
def get_proba_scores(texts):

    # Tokenize
    # NOTE: This is for just one input.
    # Let's adapt for future ones
    tokenized = tokenizer(texts, padding=True, 
                return_tensors="pt",
                truncation=True)

    # Create attention mask
    #attention_mask = (tokenized != 0).type(torch.int64)

    # Get the output logits and convert to scores
    outputs = model(**tokenized)
    probability = F.softmax(outputs.logits, dim=1).detach().numpy()
    return probability

In [ ]:
sample = 'I really did not like what I saw and I regretted every moment of it.'
probability_scores = get_proba_scores(sample)
print(probability_scores)

# Create the explainer object
explainer = LimeTextExplainer(class_names=class_names)

# Explain the instance passed in
explain_bert_output = explainer.explain_instance(
    sample, get_proba_scores, 
    num_features = 5, num_samples=250,
    top_labels = 4
)

# Show the output in notebook
explain_bert_output.show_in_notebook(text=sample)

# SHAP

This section is for interpreting models using SHAP

In [ ]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=len(labels))
masker = shap.maskers.Text(tokenizer)

In [ ]:
def score_and_visualize(text):

    # Get the labels
    prediction = pipe([text])

    # Explanation time
    explainer = shap.Explainer(pipe, masker)
    shap_values = explainer([text])
    print(shap_values.values[:,:,2])

    # Visualization
    shap.plots.bar(shap_values)
    
    return None

score_and_visualize('This was one of the best movies I have ever watched. It reminded me of my younger days and made me feel really nostalgic.')

In [4]:
import datasets
import pandas as pd

# load the emotion dataset
dataset = datasets.load_dataset("emotion", split="train")
data = pd.DataFrame({"text": dataset["text"], "emotion": dataset["label"]})

In [14]:
def f(x):

    # Get tensor values
    tv = torch.tensor(
        [
            tokenizer.encode(v, padding="max_length", max_length=128, truncation=True)
            for v in x
        ]
    )

    # Get the attention mask
    attention_mask = (tv != 0).type(torch.int64)

    # Get the outputs
    outputs = model(tv, attention_mask=attention_mask)[0].detach().numpy()

    # Get the logit scores
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T

    # Return
    val = sp.special.logit(scores)
    
    return val

In [15]:
explainer = shap.Explainer(f, tokenizer, output_names=labels)

In [ ]:
shap_values = explainer(data["text"][:3])
shap.plots.text(shap_values)

  0%|                                                                                           | 0/42 [00:00<?, ?it/s]

In [20]:
sample = ['I really did not like what I saw and I regretted every moment of it.']
shap_values = explainer(sample)
shap.plots.text(shap_values)

PartitionExplainer explainer: 2it [02:05, 125.57s/it]                                                                  
